In [1]:
%matplotlib inline
%config InlineBackend.figure_format = "retina"

import matplotlib.pyplot as plt
import seaborn

seaborn.set(context="notebook", style="darkgrid")
plt.rcParams["figure.figsize"] = (8, 6)
from tqdm.auto import tqdm, trange

from ssptools import evolve_mf_3 as emf3

import numpy as np
import scipy as sp
import astropy.units as u
from astropy.constants import c

In [2]:
m123 = [0.1, 0.5, 1.0, 100]  # Slope breakpoints for initial mass function
a12 = [-0.468, -1.178, -2.117]  # Slopes for initial mass function
nbin12 = [5, 5, 20]

# Output times for the evolution
tout = np.array([11000])

# Integration settings
N0 = 5e5  # Normalization of stars
Ndot = -0.0001  # Regulates how low mass objects are depleted default -20, 0 for 47 Tuc
tcc = 0  # Core collapse time
NS_ret = 0.1  # Initial neutron star retention
BH_ret_int = 1  # Initial Black Hole retention
BH_ret_dyn = 0.00235  # Dynamical Black Hole retention
FeHe = -0.7  # Metallicity

In [3]:
f = emf3.evolve_mf(
    m123=m123,
    a12=a12,
    nbin12=nbin12,
    tout=tout,
    N0=N0,
    Ndot=Ndot,
    tcc=tcc,
    NS_ret=NS_ret,
    BH_ret_int=BH_ret_int,
    BH_ret_dyn=BH_ret_dyn,
    FeHe=FeHe,
    natal_kicks=True,
    vesc=100,
)

/home/peter/research/ssptools/ssptools/evolve_mf_3.py:321: RuntimeWarning: divide by zero encountered in true_divide
  As = Ns / P1
/home/peter/research/ssptools/ssptools/evolve_mf_3.py:246: RuntimeWarning: divide by zero encountered in double_scalars
  Aj = Nj / self.Pk(alphaj, 1, m1, mto)


In [4]:
cs = f.Ns[-1] > 10 * f.Nmin
cr = f.Nr[-1] > 10 * f.Nmin
mj = np.r_[f.ms[-1][cs], f.mr[-1][cr]]
Mj = np.r_[f.Ms[-1][cs], f.Mr[-1][cr]]
dms = f.mes[-1][1:] - f.mes[-1][0:-1]
nms = len(f.ms[-1][cs])

In [5]:
print(mj[:nms])

[0.11851084 0.16351292 0.22560363 0.31127202 0.42947124 0.53616312
 0.6158897  0.70747148 0.80754321]


In [6]:
print("mj  | q: 0.5 \t 0.75 \t 0.95")
print("-------------------------------")
for i in range(nms):
    print(f"{mj[i]:.2f} \t {mj[i]*0.5:.3f} \t {mj[i]*0.75:.3f} \t {mj[i]*0.95:.3f}")

mj  | q: 0.5 	 0.75 	 0.95
-------------------------------
0.12 	 0.059 	 0.089 	 0.113
0.16 	 0.082 	 0.123 	 0.155
0.23 	 0.113 	 0.169 	 0.214
0.31 	 0.156 	 0.233 	 0.296
0.43 	 0.215 	 0.322 	 0.408
0.54 	 0.268 	 0.402 	 0.509
0.62 	 0.308 	 0.462 	 0.585
0.71 	 0.354 	 0.531 	 0.672
0.81 	 0.404 	 0.606 	 0.767


In [7]:
class BinaryShift:
    def __init__(self, fb, q):
        self.fb = fb
        self.q = q

    def dump(self):
        print(f"{self.fb = }")
        print(f"{self.q = }")

    def shift(self, mj, Mj):
        pass
        # here we take out mass from one bin and add it to a new bin

In [8]:
bs = BinaryShift(q=2, fb=3)

In [9]:
bs.dump()

self.fb = 3
self.q = 2
